# Variables, Constants and Libraries definition
In the command line you may use
```
az login --use-device-code --allow-no-subscriptions --output none < /dev/null
az account set --subscription subscription_id
```

In [1]:
from azure.identity import DefaultAzureCredential
credential = DefaultAzureCredential()

In [2]:
from dotenv import load_dotenv # requires python-dotenv
from azure.identity import DefaultAzureCredential, get_bearer_token_provider #requires azure-identity
from openai import AzureOpenAI
import openai, os, json, time, glob, kagglehub # openai is used for error catching, otherwise we use AzureOpenAI
import pandas as pd

if not load_dotenv("./../../config/credentials_my.env"):
    print("Environment variables not loaded, cell execution stopped")
    sys.exit()

openai_endpoint       = os.environ["AZURE_OPENAI_ENDPOINT"]
openai_api_key        = os.environ["AZURE_OPENAI_API_KEY"]
azure_deployment_name = os.environ["MODEL_DEPLOYMENT_NAME"]
openai_api_version    = os.environ["OPENAI_API_VERSION"]

credential            = DefaultAzureCredential()

token_provider        = get_bearer_token_provider(
    DefaultAzureCredential(), "https://cognitiveservices.azure.com/.default")

folder_path = "./../data"

# Create the folder if it doesn't exist
os.makedirs(folder_path, exist_ok=True)
output_file_name = os.path.join(folder_path, "questions_enriched")

# Get the current user's home directory
home_dir = os.path.expanduser("~")

print(f"openai_endpoint: {openai_endpoint}")
print(f"azure_deployment_name: {azure_deployment_name}")
print(f"openai_api_version: {openai_api_version}")

openai_endpoint: https://aiservicesiyva.openai.azure.com/
azure_deployment_name: gpt-4o
openai_api_version: 2025-04-01-preview


# [Single-Topic RAG Evaluation Dataset](https://www.kaggle.com/datasets/samuelmatsuoharris/single-topic-rag-evaluation-dataset) retrieval
There are 120 question-answer pairs in this dataset, split this way:
- 40 questions that do not have an answer within the document.
- 40 question-answer pairs that have an answer that must be generated from a single passage of the document.
- 40 question-answer pairs that have an answer that must be generated from multiple passages of the document.

In [3]:
# Define the relative path from the home directory for the Kaggle dataset
dataset_relative_path = ".cache/kagglehub/datasets/samuelmatsuoharris/single-topic-rag-evaluation-dataset/versions/3"

dataset_path = os.path.join(home_dir, dataset_relative_path)

if not os.path.exists(dataset_path):
    dataset_path = kagglehub.dataset_download("samuelmatsuoharris/single-topic-rag-evaluation-dataset") # Download latest version

print("Path to dataset files:", dataset_path)

# Find all CSV files in the directory
csv_files = glob.glob(os.path.join(dataset_path, "*.csv"))

# Load each CSV file into a DataFrame and store them in a list
dfs = [(os.path.splitext(os.path.basename(file))[0], pd.read_csv(file)) for file in csv_files]
i=0
for df in dfs:
    print(f"\n\ndataframe #{i} <{df[0]}>:")
    display(df[1].head())
    i+=1

Path to dataset files: /home/azureuser/.cache/kagglehub/datasets/samuelmatsuoharris/single-topic-rag-evaluation-dataset/versions/4


dataframe #0 <documents>:


,index,source_url,text
0,0,https://enterthegungeon.fandom.com/wiki/Bullet...,Bullet Kin\nBullet Kin are one of the most com...
1,1,https://www.dropbox.com/scl/fi/ljtdg6eaucrbf1a...,---The Paths through the Underground/Underdark...
2,2,https://bytes-and-nibbles.web.app/bytes/stici-...,Semantic and Textual Inference Chatbot Interfa...
3,3,https://github.com/llmware-ai/llmware,llmware\n\nBuilding Enterprise RAG Pipelines w...
4,4,https://docs.marimo.io/recipes.html,Recipes\nThis page includes code snippets or “...




dataframe #1 <single_passage_answer_questions>:


,document_index,question,answer
0,0,What do keybullet kin drop?,Keybullet kin drop a key upon death.
1,0,What kind of gun does the bandana bullet kin use?,The bandana bullet kin wields a machine pistol.
2,1,What do the giants look like?,"One giant is burly, grey-skinned, and 20 feet ..."
3,1,What happens on day 2?,"After a few miles of winding tunnel, you emerg..."
4,2,What were the requirements for the project?,The tool had the following requirements:\n- Ch...




dataframe #2 <no_answer_questions>:


,document_index,question
0,0,How much health does the Mutant Bullet Kin have?
1,0,Where can bishops be found?
2,1,What happened on day 10?
3,1,What did the goblins say?
4,2,Why was the H100 GPU chosen for computation?




dataframe #3 <multi_passage_answer_questions>:


,document_index,question,answer
0,0,Which enemy types wield an AK-47?,Assault-rifle wielding Bullet and Tankers wiel...
1,0,What makes jammed enemies different?,"Jammed Keybullet Kin drop 2 keys instead of 1,..."
2,1,What enemies are encountered in the second enc...,26 kobolds and 1 kobold inventor are encounter...
3,1,What monsters are encountered in this journey?,"Ropers, kobolds, kobold inventors, fire giants..."
4,2,What framework was chosen to execute the RAG p...,The LangChain framework was used to orchestrat...


In [4]:
docs_df      = dfs[0][1] # documents
ko_df        = dfs[2][1] # no_answer_questions
questions_df = pd.concat([dfs[3][1], dfs[1][1]]) # single_passage_answer_questions + multi_passage_answer_questions
docs_df["text"].to_csv(f"{folder_path}/documents.txt", index=False, header=False, sep="\n")

# Open AI client via Azure OpenAI SDK (with Responses API's)

In [5]:
openai_api_version

'2025-04-01-preview'

In [6]:
client = AzureOpenAI(
    azure_ad_token_provider = token_provider,
    api_version = openai_api_version)

response = client.responses.create(
    model = azure_deployment_name,
    input="how can I make a good pizza?")

print(f"Response: {response.output_text[:50]}...")

Response: Making a good pizza requires attention to several ...


# Create full evaluation dataset
This function adds two columns:
- `response_correct`, which is similar to `ground_truth`
- `reponse_wrong`, which contains the wrong answer, although contextualized

In [7]:
def enrich_dataset(content: dict) -> dict:

    system_message = """
    given a json dictionary whose keys are "context", "query", "ground_truth", please generate two outputs:
    1) "response_correct": another correct answer, possibly shorter and simpler, equivalent to the original one but with different words.
    2) "response_wrong": a wrong answer that is contextualized and may look correct to a person who doesn't know the document well, using the same style as the ground truth and the second truth
    """

    input = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": json.dumps(content)}
    ]
    
    text = {
        "format": {
            "type": "json_schema",
            "name": "enriched_dataset",
            "schema": {
                "type": "object",
                "properties": {
                    "response_correct": {"type": "string"},
                    "response_wrong": {"type": "string"}
                },
                "required": ["response_correct", "response_wrong"],
                "additionalProperties": False
            },
            "strict": True
        }
    }

    response = client.responses.create(
        model = azure_deployment_name,
        input = input,
        text  = text
    )

    return json.loads(response.output_text)

# Consolidate the source dataframes into a single one called `consolidated_df`

In [8]:
# Join DataFrames on docs_df.index == multi_df.document_index
consolidated_df = questions_df.merge(docs_df, left_on='document_index', right_on='index')

# Rename columns
consolidated_df = consolidated_df.rename(columns={'text': 'context', 'question': 'query', 'answer': 'ground_truth'})

# Drop unwanted columns
consolidated_df = consolidated_df.drop(columns=['index', 'source_url'])

# Reorder columns
consolidated_df = consolidated_df[['document_index', 'context', 'query', 'ground_truth']]

consolidated_df.head()

,document_index,context,query,ground_truth
0,0,Bullet Kin\nBullet Kin are one of the most com...,Which enemy types wield an AK-47?,Assault-rifle wielding Bullet and Tankers wiel...
1,0,Bullet Kin\nBullet Kin are one of the most com...,What makes jammed enemies different?,"Jammed Keybullet Kin drop 2 keys instead of 1,..."
2,1,---The Paths through the Underground/Underdark...,What enemies are encountered in the second enc...,26 kobolds and 1 kobold inventor are encounter...
3,1,---The Paths through the Underground/Underdark...,What monsters are encountered in this journey?,"Ropers, kobolds, kobold inventors, fire giants..."
4,2,Semantic and Textual Inference Chatbot Interfa...,What framework was chosen to execute the RAG p...,The LangChain framework was used to orchestrat...


# Enrich `consolidated_df` into `enriched_df`

In [9]:
# Initialize empty lists for the new columns
response_correct_list = []
response_wrong_list = []

enriched_df = consolidated_df.copy()

i = 0
total_rows = len(enriched_df)-1

import time
import openai
import json

# Iterate over the rows using a loop
for _, row in enriched_df.iterrows():
    if i >= len(response_correct_list):
        while True:
            try:
                # Call enrich_dataset() for each row
                enriched_data = enrich_dataset({
                    "context": row['context'],
                    "query": row['query'],
                    "ground_truth": row['ground_truth']
                })

                # Ensure the expected keys exist
                if 'response_correct' in enriched_data and 'response_wrong' in enriched_data:
                    response_correct_list.append(enriched_data['response_correct'])
                    response_wrong_list.append(enriched_data['response_wrong'])

                    print(f"Index {i}/{total_rows}:\n- second_truth: <{response_correct_list[i]}>\n- wrong_answer: <{response_wrong_list[i]}>\n")
                    break  # Exit loop on success
                else:
                    print("Missing keys in enriched_data, retrying...\n")
            except openai.RateLimitError:
                print("Rate limit exceeded. Waiting for 90 seconds before retrying...\n")
                time.sleep(90)  # Wait before retrying
            except json.JSONDecodeError:
                print("JSONDecodeError encountered. Retrying enrich_dataset...\n")
                continue  # Retry enrich_dataset on JSONDecodeError
    else:
        print(f"Index {i}/{total_rows} (skipping):\n- second_truth: <{response_correct_list[i]}>\n- wrong_answer: <{response_wrong_list[i]}>\n")

    i += 1



# Assign the lists as new columns in the DataFrame
enriched_df['response_correct'] = response_correct_list
enriched_df['response_wrong'] = response_wrong_list

# Display the updated DataFrame
display(enriched_df.head())

Index 0/79:
- second_truth: <Assault Bullet Kin and Tankers use AK-47s.>
- wrong_answer: <Minelets and Shroomers wield AK-47s.>

Index 1/79:
- second_truth: <Jammed Keybullet Kin give 2 keys, Jammed Chance Kin may drop double loot, and Jammed Red-Caped Bullet Kin cause contact damage. Jammed enemies move faster and escape quicker.>
- wrong_answer: <Jammed enemies always drop health, become invisible, and inflict zero damage. They are slower and more predictable than their regular counterparts.>

Index 2/79:
- second_truth: <You face 26 kobolds and a kobold inventor in the second encounter.>
- wrong_answer: <In the second encounter, you face a small group of wandering Ropers.>

Index 3/79:
- second_truth: <Ropers, kobolds, and kobold inventors.>
- wrong_answer: <Ropers, trolls, and kobold engineers.>

Index 4/79:
- second_truth: <LangChain was chosen for the RAG process, with LlamaIndex and LitGPT as initial alternatives. Llmware was considered later.>
- wrong_answer: <The LitGPT framew

,document_index,context,query,ground_truth,response_correct,response_wrong
0,0,Bullet Kin\nBullet Kin are one of the most com...,Which enemy types wield an AK-47?,Assault-rifle wielding Bullet and Tankers wiel...,Assault Bullet Kin and Tankers use AK-47s.,Minelets and Shroomers wield AK-47s.
1,0,Bullet Kin\nBullet Kin are one of the most com...,What makes jammed enemies different?,"Jammed Keybullet Kin drop 2 keys instead of 1,...","Jammed Keybullet Kin give 2 keys, Jammed Chanc...","Jammed enemies always drop health, become invi..."
2,1,---The Paths through the Underground/Underdark...,What enemies are encountered in the second enc...,26 kobolds and 1 kobold inventor are encounter...,You face 26 kobolds and a kobold inventor in t...,"In the second encounter, you face a small grou..."
3,1,---The Paths through the Underground/Underdark...,What monsters are encountered in this journey?,"Ropers, kobolds, kobold inventors, fire giants...","Ropers, kobolds, and kobold inventors.","Ropers, trolls, and kobold engineers."
4,2,Semantic and Textual Inference Chatbot Interfa...,What framework was chosen to execute the RAG p...,The LangChain framework was used to orchestrat...,"LangChain was chosen for the RAG process, with...",The LitGPT framework was ultimately selected t...


In [10]:
enriched_df.to_csv (f"{output_file_name}.csv", index=False)
enriched_df.to_json(f"{output_file_name}_records.jsonl", orient="records", lines=True)
enriched_df.to_json(f"{output_file_name}_split.json", orient="split")
enriched_df.to_json(f"{output_file_name}_table.json", orient="table")
enriched_df.to_json(f"{output_file_name}_index.json", orient="index")